# Vitessce Widget Tutorial

# Visualization of spatial single-cell RNA seq data

## 1. Import dependencies

In [3]:
import os
from os.path import join
from urllib.request import urlretrieve
import loompy

from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    LoomWrapper,
)

## 2. Download data

Download `osmFISH_SScortex_mouse_all_cells.loom` from http://loom.linnarssonlab.org/.

In [4]:
os.makedirs("data", exist_ok=True)
ds_filepath = join("data", "osmFISH_SScortex_mouse_all_cells.loom")
urlretrieve('http://loom.linnarssonlab.org/clone/osmFISH/osmFISH_SScortex_mouse_all_cells.loom', ds_filepath)

('data/osmFISH_SScortex_mouse_all_cells.loom',
 <http.client.HTTPMessage at 0x7fd49afa1a00>)

## 3. Open Loom file

In [5]:
ds = loompy.connect(ds_filepath)

## 4. Configure Vitessce

Create a Vitessce view config.

In [6]:
vc = VitessceConfig(name='Loom Example', description='osmFISH dataset of the mouse cortex including all cells')
w = LoomWrapper(ds, cell_set_attrs=["ClusterName"], embedding_attrs={"t-SNE": ["_tSNE_1", "_tSNE_2"]})
dataset = vc.add_dataset(name='SScortex').add_object(w)
tsne = vc.add_view(dataset, cm.SCATTERPLOT, mapping="t-SNE")
cell_sets = vc.add_view(dataset, cm.CELL_SETS)
spatial = vc.add_view(dataset, cm.SPATIAL)
vc.link_views([spatial], [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y], [-6.43, 10417.69, 24885.55])
vc.layout(spatial | (tsne / cell_sets));

## 5. Render the widget

A widget can be created with the `.widget()` method on the config instance. Here, the `proxy=True` parameter allows this widget to be used in a cloud notebook environment, such as Binder.

In [7]:
vw = vc.widget(proxy=True)
vw

[2020-12-31 21:57:54 -0500] [40438] [INFO] Running on http://127.0.0.1:8000 (CTRL + C to quit)


VitessceWidget(config={'version': '1.0.0', 'name': 'Loom Example', 'description': 'osmFISH dataset of the mous…